# MPP Fundamentals and Partitioning

This is Part 3 of Greenplum Database  Concepts Explained, ***MPP Fundamentals and Partitioning***. 

In [ ]:
!pip install sqlalchemy psycopg2-binary ipython-sql
!sudo apt-get install postgresql-client

import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

DB_USER   = "gpadmin"
DB_PWD    = "TRWLO7VX3SHLX"
DB_SERVER = "34.87.155.108"
DB_PORT   = "5432"
DB_NAME   = "dev"

CONNECTION_STRING=f"postgresql://{DB_USER}:{DB_PWD}@{DB_SERVER}:{DB_PORT}/{DB_NAME}"

%reload_ext sql
%sql $CONNECTION_STRING

In [ ]:
query = "SHOW gp_autostats_mode; \
ALTER DATABASE {} SET gp_autostats_mode TO 'NONE'; \
SHOW gp_autostats_mode;".format(DB_NAME)

%sql $DB_USER@$DB_NAME {''.join(query)}

## 5. MPP Fundamentals

This topic provides an overview of how a MPP (*Massively Parallel Processing*) database such as Greenplum, processes queries. Understanding this process can be useful when writing and tuning queries.

Users issue queries to Greenplum Database as they would to any database management system. They connect to the database instance on the Greenplum master host using a client application such as `psql` and submit SQL statements.

### 5.1. Understanding Parallel Query Execution & Greenplum Query Optimizer

Greenplum creates a number of database processes to handle the work of a query. On the master, the query worker process is called the **query dispatcher** (QD). The QD is responsible for creating and dispatching the query plan. It also accumulates and presents the final results. On the segments, a query worker process is called a **query executor** (QE). A QE is responsible for completing its portion of work and communicating its intermediate results to the other worker processes.

There is at least one worker process assigned to each slice of the query plan. A worker process works on its assigned portion of the query plan independently. During query execution, each segment will have a number of processes working on the query in parallel.

The Greenplum Database cost-based optimizer evaluates many strategies for executing a query and chooses the least costly method. The optimizer produces plans based on statistics generated for tables. It is important to have accurate statistics to produce the best plan. The Greenplum optimizer takes into account factors such as,
- the number of rows in tables to be joined, 
- availability of indexes, and 
- cardinality of column data when calculating the costs of alternative execution plans. 

The optimizer also accounts for the location of the data, **preferring to perform as much of the work as possible on the segments and to minimize the amount of data that must be transmitted between segments to complete the query**.

Check the following example:

### Example 1: Forcing data into a single segment

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/5-1-amzn-reviews-by-marketplace.sql
display_html('\n'.join(sqlfilecode), raw=True)

In [ ]:
query = !cat script/5-1-amzn-reviews-by-marketplace.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

#### Let's compare the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*) data distribution across segments:

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-1-data-distribution-two-tables.sql
display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/5-1-data-distribution-two-tables.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

### Let's compare the `EXPLAIN ANALYSE` output for the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*)

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-1.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-2.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-2.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

### Let's run `ANALYZE <tablename>` to collect table statistics, and compare again the `EXPLAIN ANALYSE` output for the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*)

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-3.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-3.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-4.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-4.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

## 6. Partitioning

Table partitioning enables supporting very large tables, such as fact tables, by logically dividing them into smaller, more manageable pieces. Partitioned tables can improve query performance by allowing the Greenplum Database query optimizer to scan only the data needed to satisfy a given query instead of scanning all the contents of a large table.

### 6.1. Create a new copy of the original table, define a *PARTITION* pattern (by month) and load it.

After you create the partitioned table structure, top-level parent tables are empty. Data is routed to the bottom-level child table partitions. In a multi-level partition design, only the subpartitions at the bottom of the hierarchy can contain data.

Rows that cannot be mapped to a child table partition are rejected and the load fails. To avoid unmapped rows being rejected at load time, define your partition hierarchy with a DEFAULT partition. Any rows that do not match a partition's CHECK constraints load into the DEFAULT partition.

At runtime, the query optimizer scans the entire table inheritance hierarchy and uses the CHECK table constraints to determine which of the child table partitions to scan to satisfy the query's conditions. The DEFAULT partition (if your hierarchy has one) is always scanned. DEFAULT partitions that contain data slow down the overall scan time.

When you use COPY or INSERT to load data into a parent table, the data is automatically rerouted to the correct partition, just like a regular table.

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-1-create-and-load-partition-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-1-create-and-load-partition-table.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

### 6.2. Familiarize yourself with the Partitioned Table Design and Present Basic Demographics

#### 6.2.1. Retrieve Partitioned Table Design

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-1-partition-table-design.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-1-partition-table-design.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

#### 6.2.2. Row Count per Partition

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-2-row_count_per_partition.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-2-row_count_per_partition.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

#### 6.2.3. Row Count per Partition & Segment

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-3-row-count-per-partition-segment.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-3-row-count-per-partition-segment.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

### 6.3. Partitioned Table Size and Disk Space Usage

After you create the partitioned table structure, top-level parent tables are empty. Data is routed to the bottom-level child table partitions. In a multi-level partition design, only the subpartitions at the bottom of the hierarchy can contain data.

Compare the output below with the [Non-Partitioned Table Size and Disk Usage](http://127.0.0.1:9900/notebooks/gp-demo/AWS-GP-demo-2.ipynb#4.5.2.-Using-the-gp_toolkit-Administrative-Schema-(Greenplum-5.x)).

When you use `COPY` or `INSERT` to load data into a parent table, the data is automatically rerouted to the correct partition, just like a regular table.

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-3-partitioned-table-size-disk.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-3-partitioned-table-size-disk.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

### 6.4. Verify your Partition Strategy and Demonstrate *Partition Elimination* functionality

When a table is partitioned based on the query predicate, you can use `EXPLAIN` to verify that the query optimizer scans only the relevant data to examine the query plan. For example, the `demo.amzn_reviews_2` table is date-range partitioned by year. 

### Example 2: `SELECT` data for a single day (`2011-07-12`):

The query plan for this query should show a table scan of only the following tables:

- the default partition returning 0-1 rows (if your partition design has one)
- the 2011 partition (`	demo.amzn_reviews_2_1_prt_year2011`) returning ***some number*** of rows

To confirm, execute the `EXPLAIN` query and check the query plan:

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-1.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/6-4-explain-example-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

### Example 3 : "Single-Partition" `SELECT`

### Calculate MTD (Month-to-date) Number of Reviews for the dates, up to '25 March 2015' - Original/Non-Partitioned Table (`demo.amzn_reviews`)

In [ ]:
query = !cat script/6-4-calendar-foundation.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-3-1.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/6-4-explain-example-3-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

### Calculate MTD (Month-to-date) Number of Reviews for the dates, up to '25 March 2015' - Partitioned Table (`demo.amzn_reviews_partitioned`)

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-3-2.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output= !cat script/6-4-explain-example-3-2.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

## Continue to Part 4 of Greenplum Database  Concepts Explained, **[Table Storage Models](AWS-GP-demo-4.ipynb)**.